In [55]:
import os

In [56]:
%pwd

'/Users/tapankhaladkar/Data Science/Projects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [57]:
#import os
#os.chdir("Chicken-Disease-Classification")

In [58]:
%pwd

'/Users/tapankhaladkar/Data Science/Projects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [59]:
from dataclasses import dataclass, field
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [60]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [61]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(str(config_filepath))   # Convert to string
        self.params = read_yaml(str(params_filepath))   # Convert to string

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [62]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [63]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [64]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-13 22:41:24,198: INFO: common: Created directory: artifacts]
[2025-07-13 22:41:24,200: INFO: common: Created directory: artifacts/data_ingestion]
[2025-07-13 22:41:24,201: INFO: common: Size of artifacts/data_ingestion/data.zip: 11616915 bytes]
[2025-07-13 22:41:24,201: INFO: 4037744573: File already exists of size: 11616915]
